In [2]:
# General Imports
import numpy as np
import pandas as pd
from collections import Counter

# Visualisation Imports
import matplotlib.pyplot as plt

# Scikit Imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Qiskit Imports
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC

from ipynb.fs.full.EDA import vowelData
from ipynb.fs.full.EDA import irisData
from ipynb.fs.full.EDA import breastcancerData

In [3]:
#call one of the function to load the dataset
#df_, x ,y = vowelData ()
#df_, x ,y = irisData ()
df_, x ,y = breastcancerData()


In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)[:5]
X_test = np.array(X_test)[:1]
y_train = np.array(y_train)[:5]
y_test = np.array(y_test)[:1]
num_qubits = x.shape[1]

With our training and testing datasets ready, we set up the QuantumKernel class with the ZZFeatureMap, and use the BasicAer statevector_simulator to estimate the training and testing kernel matrices.

In [8]:
zz_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement='linear', insert_barriers=True)

zz_kernel = QuantumKernel(feature_map=zz_map, quantum_instance=Aer.get_backend('statevector_simulator'))

Let's calculate the transition amplitude between the first and second training data samples, one of the entries in the training kernel matrix.

Let's createe a circuit for the first sample and try to simulate

In [9]:
zz_circuit = zz_kernel.construct_circuit(X_train[0], X_train[1])
#zz_circuit.decompose().decompose().draw(output='mpl')

We then simulate the circuit. We will use the `qasm_simulator` since the circuit contains measurements, but increase the number of shots to reduce the effect of sampling noise.

In [10]:
from qiskit.circuit.random import random_circuit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options


options = Options()
options.optimization_level = 2
options.resilience_level = 0

service = QiskitRuntimeService()
with Session(service=service, backend="ibmq_qasm_simulator") as session:
    #num_qubits = len(Counter(y))
    zz_map = ZZFeatureMap(feature_dimension=30, reps=2, entanglement='linear', insert_barriers=True)
    zz_kernel = QuantumKernel(feature_map=zz_map,quantum_instance=Aer.get_backend("qasm_simulator"))
    zz_circuit = zz_kernel.construct_circuit(X_train[0], X_train[1])
    
    sampler = Sampler(session=session, options=options)
    job = sampler.run(zz_circuit)
    result = job.result()
    
    
    
#display(circuit.draw("mpl"))
#print(f" > Quasi probability distribution: {result.quasi_dists[0]}")
print(f" > Metadata: {result.metadata[0]}")

QiskitError: 'No counts for experiment "<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1621f88b0>"'

In [11]:
zzcb_svc = SVC(kernel=zz_kernel.evaluate)
zzcb_svc.fit(X_train, y_train)
zzcb_score = zzcb_svc.score(X_test, y_test)

print(f'Callable kernel classification test score: {zzcb_score}')

In [40]:
qsvc = QSVC(quantum_kernel=zz_kernel)
qsvc.fit(X_train, y_train)
qsvc_score = qsvc.score(X_test, y_test)

print(f"QSVC classification test score: {qsvc_score}")

QSVC classification test score: 0.3787878787878788
